<a href="https://colab.research.google.com/github/DavisKerr/cse380-notebooks/blob/master/09_2_Ponder_and_Prove_Data_Compression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ponder and Prove Data Compression
#### Due: Saturday, 6 March 2021, 11:59 pm.

In [1]:
!pip install pyprimesieve

# TODO Explore Huffman Trees and Huffman Codes


Your task is examine how to compress a *special piece of information* as compactly as possible, and **calculate various compression ratios**.

Recall that the **compression ratio** of a variable-length encoding like Huffman encoding is the percentage $100(f - v)/f$, where $f$ is the number of bits per symbol of the smallest **fixed**-length encoding, and $v$ is the average number of bits per symbol with the variable-length encoding.

For example, if there were 9 different symbols in a message, $f=4$ is the number of bits of the smallest fixed-length encoding, because $2^3 = 8$ (not enough for $9$) and $2^4 = 16$ (enough and to spare). If the variable-length encoding of the message had $v=3.12$, the compression ratio would be $100(4 - 3.12)/4 \approx 22\%$.

Note that calculating the average number of bits per symbol is not strictly necessary. That's because an alternate and equivalent way is to calculate $100(ft - vt)/ft$, where $ft$ is the **total** number of bits encoded with the fixed encoding, and $vt$ is the **total** number of bits encoded with the variable-length encoding.

The *special piece of information* to be compressed is a list of the first ten million primes. This is a list that starts

|    |
|----|
|  2 |
|  3 |
|  5 |
|  7 |
| 11 |
| 13 |
| 17 |
| 19 |
| 23 |
| 29 |

  and ends

|           |
|-----------|
| 179424551 |
| 179424571 |
| 179424577 |
| 179424601 |
| 179424611 |
| 179424617 |
| 179424629 |
| 179424667 |
| 179424671 |
| 179424673 |

As ASCII text stored in a file with one prime per line, the size of this data file is slightly over 89 megabytes. The goal is to compress this down to just over 5 megabytes (5589056 bytes, to be exact). That's a 94% compression ratio!

Standard compression tools can only get about a 73% compression ratio for this ASCII data. A more clever approach is needed. Instead of compressing the list of prime numbers, compress a list of the *gaps* between them!

It doesn't save much, just the unique (occurring only once) gap size of 1 between 2 and 3, but in the spirit of de Polignac's conjecture that every *even* number appears infinitely often as a gap between consecutive primes, just consider the even-sized gaps. The result will be a list that starts with 2 (the difference between 5 and 3), 2 (the difference between 7 and 5), 4 (the difference between 11 and 7), 2 (the difference between 13 and 11), 4 (the difference between 17 and 13), 2 (the difference between 19 and 17), 4 (the difference between 23 and 19), and 6 (the difference between 29 and 23).

Generating this data is the first task. The algorithm for doing so is very straightforward:

1. Find the gaps between consecutive odd primes.
2. Store these gaps as a list of even numbers.

Tabulating the results, the first ten gaps and the last ten gaps are as follows, where the numbers after the equals signs are the gaps to list:

|                 |
|-----------------|
|  5  -   3  =  2 |
|  7  -   5  =  2 |
| 11  -   7  =  4 |
| 13  -  11  =  2 |
| 17  -  13  =  4 |
| 19  -  17  =  2 |
| 23  -  19  =  4 |
| 29  -  23  =  6 |
| 31  -  29  =  2 |
| 37  -  31  =  6 |

|                                |
|--------------------------------|
| 179424551  -  179424533  =  18 |
| 179424571  -  179424551  =  20 |
| 179424577  -  179424571  =   6 |
| 179424601  -  179424577  =  24 |
| 179424611  -  179424601  =  10 |
| 179424617  -  179424611  =   6 |
| 179424629  -  179424617  =  12 |
| 179424667  -  179424629  =  38 |
| 179424671  -  179424667  =   4 |
| 179424673  -  179424671  =   2 |

As a correctness check, see if your generated list of gaps has length 9999998.

The next step is to count how many times each gap size occurs, so that for the Huffman encoding scheme, the larger the frequency of occurrence, the smaller the number of bits encoding that gap size.

As a correctness check, here are the first ten and the last ten gap counts:

|  Gap | Count   |
|------|---------|
|    2 |  738597 |
|    4 |  738717 |
|    6 | 1297540 |
|    8 |  566151 |
|   10 |  729808 |
|   12 |  920661 |
|   14 |  503524 |
|   16 |  371677 |
|   18 |  667734 |
|   20 |  354267 |
|      |         |
|  190 |       1 |
|  192 |       3 |
|  194 |       1 |
|  196 |       1 |
|  198 |       6 |
|  202 |       2 |
|  204 |       3 |
|  210 |       4 |
|  220 |       1 |
|  222 |       1 |

Note two things from these partial gap counts:

1. Small even numbers (< 100) are well represented, larger ones (< 1000) less so.
2. Ten million primes aren't enough to have *every* even number represented; for example, 200, 206, 208, 212, 214, 216, and 218 do not appear even once.


In [2]:
from pyprimesieve import primes, primes_nth
def prime_gaps(range):
  list_of_gaps = []
  prev = 3
  gap = 0
  for prime in list(primes(4, primes_nth(range + 1))):
    gap = prime - prev
    #print(gap)
    prev = prime
    list_of_gaps.append(gap)
  return list_of_gaps

gaps = prime_gaps(10000000)
print(len(gaps))
print(gaps[0:10])
print(gaps[-10:])

9999998
[2, 2, 4, 2, 4, 2, 4, 6, 2, 6]
[18, 20, 6, 24, 10, 6, 12, 38, 4, 2]


In [3]:
gaps_uniq = set(gaps)
gap_count = list(map(lambda x: (x, gaps.count(x)), gaps_uniq))
print(gap_count)
print()

[(2, 738597), (4, 738717), (6, 1297540), (8, 566151), (10, 729808), (12, 920661), (14, 503524), (16, 371677), (18, 667734), (20, 354267), (22, 307230), (24, 453215), (26, 211203), (28, 229177), (30, 398713), (32, 123123), (34, 129043), (36, 206722), (38, 94682), (40, 111546), (42, 159956), (44, 64866), (46, 54931), (48, 93693), (50, 52183), (52, 38800), (54, 64157), (56, 32224), (58, 27985), (60, 55305), (62, 16763), (64, 17374), (66, 30960), (68, 12368), (70, 17475), (72, 17255), (74, 8540), (76, 7253), (78, 13758), (80, 6760), (82, 4791), (84, 9818), (86, 3411), (88, 3454), (90, 7056), (92, 2259), (94, 2058), (96, 3544), (98, 1831), (100, 1923), (102, 2374), (104, 1168), (106, 933), (108, 1634), (110, 941), (112, 711), (114, 1125), (116, 439), (118, 433), (120, 948), (122, 287), (124, 318), (126, 533), (128, 183), (130, 211), (132, 301), (134, 128), (136, 100), (138, 210), (140, 140), (142, 90), (144, 123), (146, 46), (148, 67), (150, 94), (152, 52), (154, 43), (156, 57), (158, 19), 

# TODO Determine Exact Size of Data to be Compressed


Without actually doing it, imagine creating an ASCII file containing the first ten million primes, represented in decimal, one prime per line. Calculate the size of this file, so you can show an exceptional compression ratio from it (see below).

Using a binary encoding instead of ASCII, each prime requires 32 bits (4 bytes), so the size of a binary file is easily determined.

Using a fixed-width encoding of the gap counts, however, requires knowing how many different gap sizes there are, after which the calculation is straightforward.

In [4]:
print(f'There are {len(set(prime_gaps(10000000)))} prime gaps total') 
# That means 7 bits are needed for a fixed length encoding.

There are 104 prime gaps total


In [5]:
prime_list = list(primes(4, primes_nth(10000000 + 1)))
bits = 0
for prime in prime_list:
  bits += (len(str(prime)) * 8)

new_line_bits = 9999999 * 8
total_bytes = (new_line_bits + bits) / 8
print(f'Total Bytes: {total_bytes}')
total_megabytes = total_bytes / 2 ** 20
print(f'Total megabytes: {total_megabytes}')

Total Bytes: 93484447.0
Total megabytes: 89.15371608734131


In [6]:
7 * 9999998

69999986

$ceil(log_{2}(104)) = 7$, which means there are 7 bits required per gap in the list of 9,999,998 prime gaps. This means it takes 69,999,986 bits for the file size. That is 8,749,999 bytes.

| Type of encoding | Size in Bytes | 
| -----------------|---------------|
| ASCII               | 93,484,447 |
| Fixed-length primes | 40,000,000 |
| Fixed-length prime gaps   | 8,749,999 |

# TODO Use Functional Python


You are encouraged to use the [anytree](https://pypi.org/project/anytree) Python library, which has a nice exporter by way of which you can graphically view trees. (You may recall using this in DM1, and thus know that **anytree** depends on [graphviz](https://graphviz.org), which you also used.)

This library uses the object-oriented features of Python to create and visualize trees. You are encouraged to use the functional features of Python as much as possible, achieving your results not by using some existing third-party libraries for building Huffman Trees and Codes, but writing your own code as cleanly and elegantly as you can.

In [7]:
def es_queue(l):
  l.sort(key=lambda x: x[0])
  if len(l) == 1:
    return l[0]
  else:
    return es_queue([[l[0][0] + l[1][0], [l[0], l[1]]]] + l[2:])

def get_tuples(l, path=''):
  if type(l[1]) is list:
    tuples_right = get_tuples(l[1][1], path+'1')
    tuples_left = get_tuples(l[1][0], path+'0')
    return tuples_left + tuples_right
  else:
    return [(l[1], path)]

In [8]:
gap_count = [[gaps.count(x),x] for x in set(gaps)]
tuples = get_tuples(es_queue(gap_count))
tree_dict = {key:value for (key, value) in tuples}

In [9]:
print(tree_dict)

{12: '000', 24: '0010', 40: '001100', 78: '001101000', 86: '00110100100', 88: '00110100101', 90: '0011010011', 58: '00110101', 66: '00110110', 96: '00110111000', 116: '00110111001000', 130: '001101110010010', 152: '00110111001001100', 162: '001101110010011010', 198: '00110111001001101100', 204: '001101110010011011010', 188: '00110111001001101101100', 190: '00110111001001101101101', 202: '0011011100100110110111', 184: '001101110010011011100', 210: '001101110010011011101', 220: '00110111001001101111000', 222: '00110111001001101111001', 194: '00110111001001101111010', 196: '00110111001001101111011', 172: '001101110010011011111', 144: '0011011100100111', 106: '0011011100101', 98: '001101110011', 76: '0011011101', 110: '0011011110000', 120: '0011011110001', 100: '001101111001', 94: '001101111010', 126: '00110111101100', 156: '00110111101101000', 148: '00110111101101001', 134: '0011011110110101', 122: '001101111011011', 114: '0011011110111', 74: '0011011111', 32: '001110', 54: '0011110', 44:

100(ft−vt)/ft , where ft is the total number of bits encoded with the fixed encoding, and vt is the total number of bits encoded with the variable-length encoding.

In [10]:
encoded = ''.join([tree_dict[x] for x in gaps])
print(len(encoded))

44712373


5589047 Bytes

In [11]:
fixed_length_prime_gaps_bytes = 8749999
fixed_length_prime_bytes = 40000000
ascii_bytes = 93484447

huffman_bytes = 5589047

fixed_gaps_ratio = 100 * ((fixed_length_prime_gaps_bytes - huffman_bytes) / (fixed_length_prime_gaps_bytes))
fixed_primes_ratio = 100 * ((fixed_length_prime_bytes - huffman_bytes) / (fixed_length_prime_bytes))
ascii_ratio = 100 * ((ascii_bytes - huffman_bytes) / (ascii_bytes))
print(f'From fixed = {fixed_gaps_ratio}') 
print(f'From binary = {fixed_primes_ratio}')
print(f'From ASCII = {ascii_ratio}') 

From fixed = 36.12516984287655
From binary = 86.0273825
From ASCII = 94.02141513443408


# TODO Achieve Target Compression Ratios

Your solution should correctly compute the following three compression ratios:

| Ratio       | Value              |
|-------------|--------------------|
| From fixed  | 36.125168653605158 |
| From binary |              86.03 |
| From ASCII  |              94.02 | 


# TODO My Report on What I Did and What I Learned

## Fun


Replace these words with your own detailing how you had fun (if you did).

## New

Replace these words with your own detailing what was something new you learned (if anything).

## Meaningful


Replace these words with your own detailing what you achieved that was meaningful or that you can build upon.

## Other

Replace this section and these words with your own describing other topics or sections of your report --- Connections, Collaborator Contributions, or anything else you feel impressed to add.

# TODO What is True?
Click on each warranted checkbox to toggle it to True (or back to False). 

NOTE: *This only works in Colab. If you run it in some other Jupyter notebook client/server environment you may have to change False to True (or vice versa) manually.*

This self-assessment is subject to revision by a grader.

In [12]:
#@markdown ## What is True about what I did?
#@markdown ### I had fun.
cb00 = False #@param {type:'boolean'}
#@markdown ### I learned something new.
cb01 = False #@param {type:'boolean'}
#@markdown ### I achieved something meaningful, or something I can build upon at a later time.
cb02 = False #@param {type:'boolean'}
#@markdown ## What is True about my report?
#@markdown ### I wrote a sufficient number of well-written sentences.
cb03 = False #@param {type:'boolean'}
#@markdown ### My report is free of mechanical infelicities.
cb04 = False #@param {type:'boolean'}
#@markdown ### I used Grammarly (or something better described in my report) to check for MIs.
cb05 = False #@param {type:'boolean'}
#@markdown ### I reported on any connections I found between these problems and something I already know. 
cb06 = False #@param {type:'boolean'}
#@markdown ### I reported who were and what contribution each of my collaborators made.
cb07 = False #@param {type:'boolean'}
#@markdown ## What is True about my calculations?
#@markdown ### I correctly calculated the number of times each gap size occurs. 
cb08 = False #@param {type:'boolean'}
#@markdown ### I correctly calculated the number of bits per gap size with a fixed encoding.
cb09 = False #@param {type:'boolean'}
#@markdown ### I correctly calculated the total number of bits encoded with the Huffman encoding.
cb10 = False #@param {type:'boolean'}
#@markdown ### I correctly calculated the total number of bits encoded with the fixed encoding.
cb11 = False #@param {type:'boolean'}
#@markdown ### I correctly calculated the compression ratio from this fixed encoding.
cb12 = False #@param {type:'boolean'}
#@markdown ### I correctly calculated the size of the first ten million primes encoded as 32-bit integer binary data.
cb13 = False #@param {type:'boolean'}
#@markdown ### I correctly calculated the compression ratio from the binary size.
cb14 = False #@param {type:'boolean'}
#@markdown ### I correctly calculated the size of the first ten million primes encoded as ASCII data.
cb15 = False #@param {type:'boolean'}
#@markdown ### I correctly calculated the compression ratio from the ASCII size (just the primes, nothing else).
cb16 = False #@param {type:'boolean'}

## DO NOT CHANGE ANYTHING IN THE NEXT CODE CELL!!
### Delete this cell and the following ones before submitting your work.

In [13]:
points_for_what_I_did = [5]*3
points_for_my_report = [8]*5
points_for_my_calculations = [5]*9
points = points_for_what_I_did + points_for_my_report + points_for_my_calculations
# cb is short for checkbox
total = sum(map(lambda n, p: p if eval(f'cb{n:02}') else 0,
                range(len(points)), points))             
total

0

# For graders

In [14]:
#@markdown ---
number_of_MIs_found = 0 #@param {type: 'slider', min: 0, max: 5}
#@markdown ---
